# VAULT satellite-ship visibility code
This Jupyter notebook demonstrates how to use the `HitFinder` class defined in `vault.py`.

## Basic usage
`vault.py` has a `HitFinder` class which automates the necesary database queries and calculations. First, import the module:

In [1]:
from vault import HitFinder

### User input
User input should include:
* The minimum altitude above the horizon for a satellite to be considered able to see a given ship
* The times to consider, as a Python `Datetime`  object
* The time window around those times, in days, to search for relevant data

Define it as a Python dictionary. Below, we define a set of parameters:
* `altitude_min` sets the minimum altitude above the horizon for a satellite and vessel considered to have line-of-sight
* `times` is a list of Python `datetime`objects representing the times of interest: May 1st through 4th, 2004
* `search_window` means we by default will fetch AIS and TLE data in a window of 7 days around `times`, i.e., from April 24th to May 11th.
* `sat_limit` limits us to fetching the first 100 unique satellite IDs we find in that time window
* `vessel_ids` specifies a set of MMSIs (vessel IDs)

`sat_ids`, not used here, would let us specify a list of satellite IDs, while `vessel_limit` would let us limit the vessels to the first N unique vessel IDs.

In [2]:
import datetime
from pprint import pprint
mmsis = ['235091871',
         '247119100',
         '311072100',
         '257310000',
         '356352000',
         '368499000',
         '366270000',
         '366988820',
         '564902000',
         '577175000']
params = {
    'altitude_min': 5.0, # degrees
    'times': [datetime.datetime(year=2004,month=1,day=d) for d in range(1,5)], # dates of interest
    'search_window': 7, # window in days around times to search for AIS/TLE data
    'sat_limit': 10000, # first 100 sats with data in window
    'vessel_limit': None, # list of specific vessel IDs
}
pprint(params)

{'altitude_min': 5.0,
 'sat_limit': 10000,
 'search_window': 7,
 'times': [datetime.datetime(2004, 1, 1, 0, 0),
           datetime.datetime(2004, 1, 2, 0, 0),
           datetime.datetime(2004, 1, 3, 0, 0),
           datetime.datetime(2004, 1, 4, 0, 0)],
 'vessel_limit': None}


Set up a `HitFinder` instance, which parses the `params` dict, does some preprocessing, and assigns sensible defaults for any missing data:

In [3]:
hitFinder = HitFinder(params)
pprint(hitFinder.get_params())

Parsing input...
{'altitude_min': 5.0,
 'sat_ids': None,
 'sat_limit': 10000,
 'search_window': Timedelta('7 days 00:00:00'),
 'times': [Timestamp('2004-01-01 00:00:00+0000', tz='UTC'),
           Timestamp('2004-01-02 00:00:00+0000', tz='UTC'),
           Timestamp('2004-01-03 00:00:00+0000', tz='UTC'),
           Timestamp('2004-01-04 00:00:00+0000', tz='UTC')],
 'tmax': Timestamp('2004-01-11 00:00:00+0000', tz='UTC'),
 'tmin': Timestamp('2003-12-25 00:00:00+0000', tz='UTC'),
 'vessel_ids': None,
 'vessel_limit': None}


Now open the database connection:

In [4]:
%time hitFinder.open_db_connection(host='vault-mysql2.conyv1ix7bv7.us-east-1.rds.amazonaws.com',user='admin',password='vault2021!')

Opening DB connection...
CPU times: user 3.3 ms, sys: 2.63 ms, total: 5.93 ms
Wall time: 195 ms


The `hitFinder.db` class member has a `query` method to query the database. As an example, we can inspect the time ranges spanned by the AIS (`vault.ais`) and TLE (`vault.tle`) databases:

In [5]:
%time hitFinder.db.query('select max(base_date_time),min(base_date_time) from vault.ais')

CPU times: user 1.29 ms, sys: 1.47 ms, total: 2.75 ms
Wall time: 18 s


[{'max(base_date_time)': datetime.datetime(2017, 1, 31, 23, 59, 58),
  'min(base_date_time)': datetime.datetime(2008, 12, 31, 23, 58, 59)}]

In [6]:
%time hitFinder.db.query('select max(timestamp),min(timestamp) from vault.tle')

CPU times: user 1.06 ms, sys: 1.06 ms, total: 2.12 ms
Wall time: 15.3 s


[{'max(timestamp)': datetime.datetime(2004, 8, 18, 20, 0),
  'min(timestamp)': datetime.datetime(1959, 1, 11, 1, 49, 23)}]

Once the database is open, we can fetch the data. The `load_tle_data()` and `load_ais_data()` methods handle this; you can specify a subset of satellite or vessel IDs via the `ids` keyword, as well as different time limits via `tmin` and `tmax`. By default these are taken from `params`; if the satellite/vessel IDs are not specified, the routine will fetch all data between `params['tmin']` and `params['tmax']`.

As an example below, we extend the timeframe for the AIS data to 2015, as at time of writing there was no AIS data that overlapped with TLE data:

In [7]:
%time hitFinder.load_tle_data()

Fetching TLE data for timeframe and satellite IDs
Fetching satellite IDs
CPU times: user 8.15 s, sys: 312 ms, total: 8.46 s
Wall time: 1min 7s


In [9]:
%time hitFinder.load_ais_data(tmin=datetime.datetime(year=2015,month=1,day=1),tmax=datetime.datetime(year=2015,month=1,day=2))

Fetching AIS data for timeframe and vessel IDs
Fetching vessel IDs
select * from vault.ais where base_date_time between "2015-01-01T00:00:00" and "2015-01-02T00:00:00"
CPU times: user 4.43 s, sys: 140 ms, total: 4.56 s
Wall time: 54.6 s


The loaded data is stored as Pandas dataframes under the `HitFinder.tle_df` and `HitFinder.ais_df` class members:

In [12]:
hitFinder.tle_df.describe()

,satellite_number,calc_anomaly,calc_mean_motion,calc_ballistic_coefficient,calc_inclination,line_number,calc_perigee,calc_drag,calc_right_ascension,calc_mean_motion_2,calc_eccentricity
count,122471.000000,122471.000000,122471.000000,1.224710e+05,122471.000000,1.224710e+05,122471.000000,122471.000000,122471.000000,122471.000000,122471.000000
mean,15650.285063,181.886964,11.744055,1.646424e-04,73.930916,1.807664e+06,177.807275,0.000511,167.939608,0.000002,0.077484
std,8277.592211,108.895297,4.178560,6.988131e-03,25.569300,2.349923e+06,103.078177,0.013831,105.479120,0.000415,0.187019
min,5.000000,0.000200,0.242778,-4.791730e-01,0.001300,5.641000e+03,0.001500,-3.860100,0.000600,-0.000028,0.000003
25%,8566.000000,83.151350,12.325100,-2.000000e-08,65.779100,6.689000e+04,87.522250,0.000100,73.128300,0.000000,0.002144
50%,16202.000000,185.070000,13.534900,1.000000e-06,75.105500,1.281290e+05,175.710000,0.000100,157.753000,0.000000,0.006184
75%,22676.000000,278.328500,14.160000,9.295000e-06,97.732500,4.994634e+06,267.955000,0.000418,258.675000,0.000000,0.022309
max,28145.000000,359.999000,16.453600,1.000000e+00,144.645000,5.055931e+06,359.999000,0.825060,359.998000,0.119890,0.884896


In [13]:
hitFinder.ais_df.describe()

,heading,length,lon,sog,draft,vessel_type,width,cog,cargo,lat
count,81434.000000,81420.000000,81434.000000,81434.000000,81420.000000,81420.000000,81420.000000,81434.000000,67183.000000,81434.000000
mean,360.736928,64.708333,-166.355635,2.327740,3.037472,892.335298,14.627520,-38.745261,31.178617,54.130109
std,176.897329,73.550358,2.225055,4.669343,4.420976,309.937091,13.546394,116.595225,26.366108,0.628096
min,0.000000,0.000000,-179.765000,-0.100000,-12.800000,0.000000,0.000000,-204.800000,0.000000,51.731100
25%,222.000000,30.000000,-166.573000,0.000000,0.000000,1001.000000,8.530000,-133.600000,1.000000,53.869000
50%,511.000000,39.630000,-166.539000,0.000000,0.000000,1001.000000,9.760000,-80.400000,30.000000,53.901100
75%,511.000000,56.390000,-166.508000,0.200000,4.570000,1001.000000,12.200000,49.100000,39.000000,54.212350
max,511.000000,349.000000,-162.001000,22.000000,18.200000,1025.000000,70.000000,204.700000,152.000000,57.188400


Now just call the `find_all_hits()` method. This uses the array of times from `params` and returns a list of either Python dictionaries or JSON strings suitable for, e.g., sending to an ElasticSearch instance. Here, we set `json=False` so we can inspect the resulting dictionary better:

In [15]:
%time hits = hitFinder.find_hits(t=hitFinder.get_params()['times'][0],json=False)

CPU times: user 2min 2s, sys: 428 ms, total: 2min 2s
Wall time: 2min 2s


In [26]:
import json
with open('all_sats_all_ships.json','w+') as fp:
    fp.write(json.dumps(hits))

Each element of the resulting list has a number of keys:
* `utc`: the time requested in ISO format.
* `satellites`: Information on each satellite at this time
    * `id`: Identifier  (satellite catalog number)
    * `lat`: Latitude (degrees)
    * `lon`: Longitude (degrees)
    * `alt`: Altitude (kilometers)
    * `horizon`: Estimated distance in degrees to horizon along Earth's surface from satellite's point of view. Assumes a spherical Earth.
* `vessels`: Information on each vessel at this time
    * `id`: Identifier (MMSI)
    * `lat`: Latitude (degrees)
    * `lon`: Longitude (degrees)
    * `alt`: Altitude (kilometers)
    * `sog`: Speed over ground (km/hr)
    * `cog`: Course over ground (degrees from true North)
    * `method`: Method used to estimate ship position (`interp` = interpolation between adjacent AIS entries, `extrap` = extrapolation from nearest AIS entry)
    * `nearest_ais`: List of AIS entres for this vessel immediately before and immediately after this time.
    * `delta_t`: Time (in hours) between this time and the closest AIS entry
* `pairs`: Information on satellite-vessel pairs, indexed by strings of the form `<satellite_id>,<vessel_id>`
    * `sat_id`: Satellite identifier (catalog number)
    * `ves_id`: Vessel identifier (MMSI)
    * `azimuth`: Azimuthal angle from vessel to satellite (degrees)
    * `elevation`: Elevation of satellite above vessel's horizon (degrees)
    * `range`: Distance between satellite and vessel (kilometers)
    * `hit`: Boolean, true if elevation > `min_degrees`, false otherwise
    
Below, we look at each top-level key of the first element of `hits`:

In [ ]:
pprint(hits[0]['utc'])

In [ ]:
pprint(hits[0]['satellites'])

In [ ]:
pprint(hits[0]['vessels'])

In [ ]:
pprint(hits[0]['pairs'])